In [1]:
ls -lh data/

Invalid switch - "".


In [2]:
%matplotlib inline
import pandas as pd

# Loading The CSV File

In [3]:
pd.to_datetime("2018-1-1 13:20")

Timestamp('2018-01-01 13:20:00')

In [4]:
pd.to_timedelta("08:30:00")

Timedelta('0 days 08:30:00')

In [5]:
%%time 
col_names=["frame", "x", "y", "obj","size","seq","tbd1","tbd2","tbd3","filename","time","path_time","delta_time","tbd4"]
use_cols=["frame", "x", "y", "obj","size","seq","filename","time","delta_time"]
df=pd.read_csv('data/fixed.csv',  names = col_names,usecols=use_cols, parse_dates=['time'])

df['time']= df['time'] +pd.to_timedelta(df['delta_time'])
df=df.drop(['delta_time'],axis=1)

Wall time: 20 s


In [6]:
len(df)

2997979

In [7]:
df.head(10)

,frame,x,y,obj,size,seq,filename,time
0,0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
1,0,4,291,1,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
2,0,629,7,2,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
3,0,253,4,3,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
4,0,185,4,4,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
5,1700,5,102,5,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
6,1701,6,102,5,362,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
7,1702,6,102,5,362,2,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
8,1703,7,102,5,362,3,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17
9,1704,9,102,5,362,4,14-16\20170817012709_100.h264.txt,2017-08-17 01:28:17


In [8]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
frame       int64
x           int64
y           int64
obj         int64
size        int64
seq         int64
filename    object
time        datetime64[ns]
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 337.3 MB


In [9]:
df.describe()

,frame,x,y,obj,size,seq
count,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06,2.997979e+06
mean,8.289598e+03,2.730162e+02,1.480890e+02,5.053650e+01,4.675812e+07,1.158957e+02
std,5.189150e+03,1.609973e+02,7.650773e+01,8.449936e+01,3.391170e+07,1.502103e+02
min,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00
25%,3.712000e+03,1.440000e+02,9.500000e+01,1.200000e+01,1.419073e+07,1.600000e+01
50%,8.119000e+03,2.690000e+02,1.340000e+02,2.800000e+01,4.510348e+07,5.900000e+01
75%,1.263600e+04,4.020000e+02,2.050000e+02,5.100000e+01,5.399656e+07,1.620000e+02
max,1.847600e+04,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03


In [10]:

# file_name_output = "fixed_without_dups.csv"

# # Notes:
# # - the `subset=None` means that every column is used 
# #    to determine if two rows are different; to change that specify
# #    the columns as an array
# # - the `inplace=True` means that the data structure is changed and
# #   the duplicate rows are gone  
# df.drop_duplicates(subset=None, inplace=True)

# # Write the results to a different file
# df.to_csv(file_name_output)



In [11]:
len(df)

2997979

In [12]:

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')



In [13]:
optimized_df = df.copy()

optimized_df[converted_int.columns] = converted_int

In [14]:
df[converted_int.columns] = converted_int

df.info(memory_usage='deep')
df_obj = df.select_dtypes(include=['object']).copy()
df_obj.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
frame       uint16
x           uint16
y           uint16
obj         uint16
size        uint32
seq         uint16
filename    object
time        datetime64[ns]
dtypes: datetime64[ns](1), object(1), uint16(5), uint32(1)
memory usage: 240.1 MB


,filename
count,2997979
unique,601
top,14-16\20170817015738_100.h264.txt
freq,120374


In [15]:
converted_obj = pd.DataFrame()

for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]
# print(mem_usage(df_obj))
# print(mem_usage(converted_obj))

# compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
# compare_obj.columns = ['before','after']
# compare_obj.apply(pd.Series.value_counts)

In [16]:
df[converted_obj.columns] = converted_obj
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
frame       uint16
x           uint16
y           uint16
obj         uint16
size        uint32
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(5), uint32(1)
memory usage: 68.7 MB


In [17]:
df.head()

,frame,x,y,obj,size,seq,filename,time
0,0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
1,0,4,291,1,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
2,0,629,7,2,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
3,0,253,4,3,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
4,0,185,4,4,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09


In [18]:
%time df.to_pickle('data/paths.pkl.xz')

Wall time: 24.3 s


In [19]:
%time pickle=pd.read_pickle('data/paths.pkl.xz')
pickle.info()

Wall time: 1.32 s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997979 entries, 0 to 2997978
Data columns (total 8 columns):
frame       uint16
x           uint16
y           uint16
obj         uint16
size        uint32
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(5), uint32(1)
memory usage: 68.6 MB


In [20]:
import os
os.path.getsize('data/paths.pkl.xz')/1000000


3.537672